###  https://github.com/jonbruner/tensorflow-basics/blob/master/save-load/save.ipynb

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

## PUT MODELS IN DIFFERENT FILES AND SAVE THEM
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pylab
from random import randint


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
train_digits, train_labels = mnist.train.images[:], mnist.train.labels[:]
test_digits, test_labels = mnist.test.images[:], mnist.test.labels[:]

In [3]:
def data_iterator(x, y):
    """ 
    A helper function to create batches for the training data 
    IN: 
        (1) np array: the training images
        (2) np array: the labels [0000010000]
    OUT:
        Batches to train on using iter_.next()
    """
    batch_idx = 0
    while True:
        # shuffle labels and features
        idxs = np.arange(0, len(x))
        np.random.shuffle(idxs)
        shuf_features = x[idxs]
        shuf_labels = y[idxs]
        batch_size = 100
        for batch_idx in range(0, len(x), batch_size):
            images_batch = shuf_features[batch_idx:batch_idx+batch_size]
            images_batch = images_batch.astype("float32")
            labels_batch = shuf_labels[batch_idx:batch_idx+batch_size]
            yield images_batch, labels_batch      

In [4]:
sess = tf.InteractiveSession()

# ONE LAYER

In [5]:
# ONE LAYER
iter_ = data_iterator(train_digits, train_labels)      
# MODEL HERE 
x = tf.placeholder(tf.float32, [None, 784], name="input")
y_ = tf.placeholder(tf.float32, [None, 10])
# 10 weights to learn
w1 = tf.Variable(tf.zeros([784, 10]))
# 10 biases to learn
b1 = tf.Variable(tf.zeros([10]))    
y = tf.nn.softmax(tf.matmul(x, w1) + b1, name="output")

init = tf.initialize_all_variables()
sess.run(init)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# use gradient descent just because.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init = tf.initialize_all_variables()
sess.run(init)

for _ in range(1000):
    batch_xs, batch_ys = iter_.next()  #mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

final = sess.run(accuracy, feed_dict={x: test_digits, y_: test_labels})
print final

saver = tf.train.Saver()
save_path = saver.save(sess, "./models/one_layer.ckpt")
print("Model saved to %s" % save_path)
tf.reset_default_graph()
sess = tf.InteractiveSession()


0.9194
Model saved to ./models/one_layer.ckpt


# RELU 

In [12]:
iter_ = data_iterator(train_digits, train_labels)      
# could also change this to 784, 256 ->  256, 256-> 256, 10
# http://stackoverflow.com/questions/41993311/adding-more-layers-to-tensorflow-mnist-tutorial-makes-accuracy-drop                          
# but that would be unfair comparison
# relu init http://cs231n.github.io/neural-networks-2/ -> relu bias 

x = tf.placeholder(tf.float32, [None, 784], name="input")

W = tf.Variable(tf.random_normal([784, 256], stddev=0.1)) 
W1 = tf.Variable(tf.random_normal([256, 256], stddev=0.1))
W2 = tf.Variable(tf.random_normal([256, 10], stddev=0.1))
b = tf.Variable(tf.zeros([256]))
b1 = tf.Variable(tf.zeros([256]))
b2 = tf.Variable(tf.zeros([10]))

y = tf.matmul(x, W) + b
y = tf.nn.relu(y)
y = tf.matmul(y, W1) + b1
y = tf.nn.relu(y)
y = tf.nn.softmax(tf.matmul(y, W2) + b2, name="output")

y_ = tf.placeholder(tf.float32, [None, 10])

init = tf.initialize_all_variables()
sess.run(init)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# use gradient descent just because.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# evals
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.initialize_all_variables()
sess.run(init)

for _ in range(1000):
    batch_xs, batch_ys = iter_.next()  #mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})


final = sess.run(accuracy, feed_dict={x: test_digits, y_: test_labels})
print final

saver = tf.train.Saver()
save_path = saver.save(sess, "./models/relu.ckpt")
print("Model saved to %s" % save_path)
tf.reset_default_graph()
sess = tf.InteractiveSession()


0.9694
Model saved to ./models/relu.ckpt


# SQUARED

In [13]:

iter_ = data_iterator(train_digits, train_labels)      
# MODEL HERE 
##
x = tf.placeholder(tf.float32, [None, 784], name="input")
# 10 weights to learn
w1 = tf.Variable(tf.zeros([784, 10]))
# 10 biases to learn
b1 = tf.Variable(tf.zeros([10]))    
y = tf.nn.softmax(tf.matmul(x, w1) + b1, name="output")

y_ = tf.placeholder(tf.float32, [None, 10])

init = tf.initialize_all_variables()
sess.run(init)

'''THIS CHANGES FROM THE ONE LAYER MODEL'''
cross_entropy = tf.reduce_mean(tf.square(y - y_))
##
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.initialize_all_variables()
sess.run(init)

for _ in range(1000):
    batch_xs, batch_ys = iter_.next()  #mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

final = sess.run(accuracy, feed_dict={x: test_digits, y_: test_labels})
print final

saver = tf.train.Saver()
save_path = saver.save(sess, "./models/one_layer_sqr.ckpt")
print("Model saved to %s" % save_path)
tf.reset_default_graph()
sess = tf.InteractiveSession()

0.8916
Model saved to models/one_layer_sqr.ckpt


In [14]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

iter_ = data_iterator(train_digits, train_labels)      

x = tf.placeholder(tf.float32, shape=[None, 784], name="input")
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32, name="keep_prob")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name="output")

init = tf.initialize_all_variables()
sess.run(init)

cross_entropy = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.initialize_all_variables()
sess.run(init)

# train 1000 batches of size 100 (defined in iterator function)
for _ in range(1000):
    batch_xs, batch_ys = iter_.next() 
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    
# evaluation
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
final = sess.run(accuracy, feed_dict={x: test_digits, y_: test_labels, keep_prob: 1.0})
print final

saver = tf.train.Saver()
save_path = saver.save(sess, "./models/conv.ckpt")
print("Model saved to %s" % save_path)
tf.reset_default_graph()
sess = tf.InteractiveSession()


0.9568
Model saved to models/conv.ckpt
